In [1]:
pwd

'/Users/ozkansafak/code/fraktur/notebooks'

In [2]:
%%time
%load_ext autoreload
%autoreload 2

# import standard Python modules
import time 
import numpy as np
import os, sys
import json
import re
import glob
import openai

# # Get the root path of the project
sys.path.append(os.path.abspath(".."))

# Import necessary modules from your src directory
from src.utils import timeit, encode_image, plt
from src.processing import compute_log_spectrum_1d, extract_image_bbox, save_images
from src.api_requests import construct_payload, send_gpt_request
from src.document_generation import save_document
from src.api_requests import single_page

from IPython.core.display import display, HTML
from IPython.display import clear_output
display(HTML("<style>.container { width:90% !important; }</style>"))

print('sys.executable:', sys.executable)
print('sys.version:', sys.version, '\n')

<timed exec>:23: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display


sys.executable: /Users/ozkansafak/code/fraktur/.venv/bin/python3
sys.version: 3.10.9 (main, Mar  1 2023, 12:20:14) [Clang 14.0.6 ] 

CPU times: user 382 ms, sys: 71.7 ms, total: 454 ms
Wall time: 1.08 s


---
## Fraktur Translator

In [35]:
# # Rename Split pdfs
# foldername = "Pages from Der Weltkrieg v8 chap 4A"
# pageno = 103

# pdfpaths = glob.glob(f"../data/{foldername}/*.pdf")

# for i, pdfpath in enumerate(sorted(pdfpaths)):
#     pdfname = pdfpath.split("/")[-1]
#     print(pdfname)
#     newname = "/".join(pdfpath.split("/")[:-1])
#     print(newname + "/" + f'page_{str(i+pageno)}.pdf')
    
#     os.rename(pdfpath, newname + "/" + f'page_{str(i + 103)}.pdf' )
    

Pages from Der Weltkrieg v8 chap 4A-01.pdf
../data/Pages from Der Weltkrieg v8 chap 4A/page_103.pdf
Pages from Der Weltkrieg v8 chap 4A-02.pdf
../data/Pages from Der Weltkrieg v8 chap 4A/page_104.pdf
Pages from Der Weltkrieg v8 chap 4A-03.pdf
../data/Pages from Der Weltkrieg v8 chap 4A/page_105.pdf
Pages from Der Weltkrieg v8 chap 4A-04.pdf
../data/Pages from Der Weltkrieg v8 chap 4A/page_106.pdf
Pages from Der Weltkrieg v8 chap 4A-05.pdf
../data/Pages from Der Weltkrieg v8 chap 4A/page_107.pdf
Pages from Der Weltkrieg v8 chap 4A-06.pdf
../data/Pages from Der Weltkrieg v8 chap 4A/page_108.pdf
Pages from Der Weltkrieg v8 chap 4A-07.pdf
../data/Pages from Der Weltkrieg v8 chap 4A/page_109.pdf
Pages from Der Weltkrieg v8 chap 4A-08.pdf
../data/Pages from Der Weltkrieg v8 chap 4A/page_110.pdf
Pages from Der Weltkrieg v8 chap 4A-09.pdf
../data/Pages from Der Weltkrieg v8 chap 4A/page_111.pdf
Pages from Der Weltkrieg v8 chap 4A-10.pdf
../data/Pages from Der Weltkrieg v8 chap 4A/page_112.pdf


In [56]:
# Set the OpenAI API key from environment variables
openai.api_key = os.getenv("OPENAI_API_KEY")

# Set the headers for the API request
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {openai.api_key}"
}

# Select the value of model_name in ["gpt-4o-2024-08-06", "gpt-4o-mini-2024-07-18"]
model_name = "gpt-4o-2024-08-06"
plotter = False

# Folder containing the Fraktur PDF files
image_path = "../data/Pages from Der Weltkrieg v8 chap 4A/*pdf"

# Get the list of PDF file paths
fnames = sorted(glob.glob(image_path))
german_texts = {}
english_texts = {}
total_time = 0 

In [58]:
for i, fname in enumerate(fnames[1:]):
    start = time.time()

    pageno = re.search(r'page_(.*?)\.pdf', fname, re.DOTALL).group(1)
    print(f"\n{'>'*10} page:{pageno} (i:{i}/{len(fnames)-1}) ", end="")
    
    raw_text, raw_german_text, german_text, english_text = single_page(fname, model_name, headers, plotter, pageno)
    
    german_texts[pageno] = german_text
    english_texts[pageno] = english_text
    delta = time.time() - start 
    total_time += delta
    print(f"[time:{delta:.2f}sec] [TotalRuntime:{int(total_time//60)} mins {(total_time - int(total_time//60)*60):.2f} sec] {'>'*10}\n")
    print(raw_text)



>>>>>>>>>> page:104 (i:0/35) [time:38.02sec] [TotalRuntime:0 mins 38.02 sec] >>>>>>>>>>

<raw_german>
Die Front des Oberbefehlshabers Ost bis zum 2. Juli.
Generalfeldmarschall Leopold Prinz von Bayern den Oberbefehl. Er verfügte insgesamt über 11½ Divisionen Infanterie, davon 7½ aktive und Reserve-Divisionen, und zwei Kavallerie-Divisionen. Diese Kräfte standen schon seit den Dezemberkämpfen auf etwa 100 Kilometer Frontbreite in engster Berührung mit dem Gegner (russische 5. und 2. Armee), der, auf etwa 15 Divisionen geschätzt, in gut ausgebauten zusammenhängenden Stellungen gegenüberlag.
Rechts der Weichsel von Plock bis zur Szwa nördlich von Ostrolenka deckte die Armee-Gruppe Gallwitz in Stärke von 12½ Divisionen Infanterie, davon 7½ aktive und Reserve-Divisionen, und zwei Kavallerie-Divisionen eine Frontbreite von rund 260 Kilometern. Auch die Aufstellung des Feindes war hier lichter, der Abstand von ihm größer als bei der 9. Armee. Man rechnete mit reichlich 14 Divisionen der russ

In [59]:
# save json outputs
folder_name = 'Chapter_4A'

if not os.path.exists(f'../output/{folder_name}'):
    os.makedirs(f'../output/{folder_name}')

with open(f'../output/{folder_name}/english_texts.json', 'w') as f:
    json.dump(english_texts, f)
with open(f'../output/{folder_name}/german_texts.json', 'w') as f:
    json.dump(german_texts, f)

doc1, fname1 = save_document(german_texts, folder_name, language='German')
doc2, fname2 = save_document(english_texts, folder_name, language='English')


``` 
1.  Upload Input folder of pdfs to blob storage.
2.  Read file from s3.
3.  FFT in y -> (x_hi, x_lo), write half_cropped_image to s3
4.  FFT in x -> (y_hi, y_lo), write cropped_image to s3
5.  Read cropped image from s3 -> encode_image -> translate and transcribe -> JSON output

```

### Available models and pricing:
```
"gpt-4o-2024-08-06":
    "price_txt": "$2.50 / 1M input tokens"
    "price_img": "$0.001913 / 1500px^2"
    
"gpt-4o-mini-2024-07-18":
    "price_txt": "$0.150 / 1M input tokens"
    "price_img": "$0.003825 / 1500px^2"
    
```